In [2]:
"""
Pointer Immediate Tasks:
- Run an experiment where I take trained pointer networks and transition them to a dominoe based value 
function and a gamma < 1, and show that they can learn to prioritize playing high value dominoes first. Then...
-  Add the context vector that encodes the number of turns left (with uncertainty?)
    - so the full pointer network will get an extra context input that describes how many turns are left
    - 0 rewards will be given after the possible turns are over
    - so the network will have to learn to get as much value out as quickly as possible
-  Also apply these networks to the vehicle routing problem?
-  Analyze encoding space of pointer networks...
-  Do the encoder swap of different pointer layers...
-  Does the speed of learning for the different networks on the sequencer task come from true performance or just sensitivity to the temperature? 
"""

"""
TODO For refactoring
at the moment there is no flexibility on the loss function control for the supervised dataset!!!!

Add mechanism for printing the arguments used to build a pointernetwork so the user can see what they did. 
Add mechanism for storing hidden parameters to entire pointer network

Add documentation of baseline updates and performance etc
Add some dataset specific summary plots and integrate into plotting code? 
Get the supervised learning methods working for each dataset and task
Checkpointing, figure making, logging, etc

it worked!!! now trying without embedding bias...
it works without embedding bias. It works (with different speeds per pointer layer!) with lower train temperature
 (but of course that could be because of differential sensitivity to temperature..., should test that directly)
now trying with 1 encoding layer. 

:)


TODO: 
DOMINOES SEQUENCER Comparison of max to real reward:
- Add target to batch (can do post-hoc, even if not requested)
- Measure reward of target
- Add a 2D vector comparing max and real reward for each batch element!!

TSP Distance Traveled:
- Explicitly measure the distance traveled by the agent in the TSP task
- Compare to Held-Karp Solution


TODO ASAP!!!!!!
- Get test result plots in there for good plotting, then start running experiments with different parameters
  so you can save / see the results!!!
"""

print('')

In [2]:
%reload_ext autoreload
%autoreload 2

from time import time
import torch
from ptrseq.datasets import get_dataset

In [15]:
task = "tsp"
dataset = get_dataset(task, build=True, num_cities=10)
batch = dataset.generate_batch(batch_size=4, return_target=True)

for k in batch.keys():
    print(k)

input
dists
init
num_cities
coord_dims
batch_size
return_target
ignore_index
threads
target


In [5]:
batch["target"]

tensor([[9, 7, 4, 2, 5, 6, 1, 0, 8, 3],
        [1, 9, 5, 6, 2, 0, 4, 8, 3, 7],
        [6, 3, 1, 9, 2, 0, 7, 5, 8, 4],
        [7, 5, 3, 9, 1, 6, 8, 2, 4, 0]])

In [4]:
reward = dataset.reward_function(batch["target"], batch)

ib = 0
prev = batch["init"][ib]
for i, s in enumerate(batch["target"][ib]):
    if i==(batch["target"].size(1)-1):
        add_dist = batch["dists"][ib][s, batch["init"][ib]]
    else:
        add_dist = 0
    print(f"From {prev} to {s} with reward {-reward[ib][i]:.2f}, distance: {batch['dists'][ib][prev, s] + add_dist :.2f}")
    prev = s

From 7 to 6 with reward 0.15, distance: 0.15
From 6 to 9 with reward 0.32, distance: 0.32
From 9 to 0 with reward 0.34, distance: 0.34
From 0 to 5 with reward 0.57, distance: 0.57
From 5 to 8 with reward 0.29, distance: 0.29
From 8 to 3 with reward 0.28, distance: 0.28
From 3 to 2 with reward 0.56, distance: 0.56
From 2 to 4 with reward 0.48, distance: 0.48
From 4 to 1 with reward 0.40, distance: 0.40


In [2]:
highest_dominoe = 9
dataset = DominoeDataset("sequencer", highest_dominoe, hand_size=8, return_target=True)

batch = dataset.generate_batch(train=False, batch_size=4, value_method="length")
dominoes = dataset.get_dominoe_set(train=False)

target_as_choice = batch["target"].clone()
target_as_choice[target_as_choice==-1] = dataset.prms["hand_size"]

reward, direction = dataset._measurereward_sequencer(target_as_choice, batch, return_direction=True)
print(reward)

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.]])


In [7]:
ib = 1
hand = dominoes[batch["selection"][ib]]
hand_null = torch.cat([hand, -torch.ones((1, 2))], dim=0)

print(batch["available"][ib])
for c, d in zip(target_as_choice[ib], direction[ib]):
    print(hand_null[c], "forward" if d==0 else "reverse")

tensor(8)
tensor([6., 8.]) reverse
tensor([2., 6.]) reverse
tensor([2., 8.]) forward
tensor([1., 8.]) reverse
tensor([1., 4.]) forward
tensor([-1., -1.]) reverse
tensor([-1., -1.]) reverse
tensor([-1., -1.]) reverse
tensor([-1., -1.]) reverse
